In [1]:
import torch
import torchvision

In [6]:
folder="./dataset"
data_train=torchvision.datasets.FashionMNIST(root=folder,train=True,download=True)
data_test=torchvision.datasets.FashionMNIST(root=folder,train=False,download=True)

100.0%


Extracting ./dataset/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./dataset/FashionMNIST/raw



100.0%


Extracting ./dataset/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./dataset/FashionMNIST/raw



100.0%


Extracting ./dataset/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./dataset/FashionMNIST/raw



100.0%

Extracting ./dataset/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./dataset/FashionMNIST/raw



In [15]:
data_test.targets.shape

torch.Size([10000])